# Text Analysis Group Project

by: Shan Ali

In [1]:
# load libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# define top 100 dataframe
top100 = pd.DataFrame(np.zeros((500,3)),columns=['Year','Title','Artist'])
dates = ['2015','2016','2017','2018','2019'] # can go to 2002

for date in dates:
    url = 'https://www.billboard.com/charts/year-end/' + date + '/hot-r-and-and-b-hip-hop-songs/'
    
    # scrape lyrics
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    title = html.find_all('div', class_='ye-chart-item__title')
    artist = html.find_all('div', class_='ye-chart-item__artist')

    for i in range(100):
        j = c+i
        top100.iloc[j,0] = date
        try:
            top100.iloc[j,1] = title[i].get_text().replace('\n', '')
            art = artist[i].get_text().replace('\n', '')
            if len(art.split(' Featuring')) > 1:
                top100.iloc[j,2] = art.split(' Featuring')[0]
            elif len(art.split(',')) > 1:
                top100.iloc[j,2] = art.split(',')[0]
            elif len(art.split(' &')) > 1:
                top100.iloc[j,2] = art.split(' &')[0]
            elif len(art.split(' X')) > 1 and art.split(' X')[0] != 'Lil Nas':
                top100.iloc[j,2] = art.split(' X')[0]
            else:
                top100.iloc[j,2] = art
        except:
            top100.iloc[j,1] = np.nan
            top100.iloc[j,2] = np.nan
    c += 100
    
# clean and display data
top100 = top100.dropna().reset_index(drop=True)
top100

,Year,Title,Artist
0,2015,See You Again,Wiz Khalifa
1,2015,Trap Queen,Fetty Wap
2,2015,Watch Me,Silento
3,2015,The Hills,The Weeknd
4,2015,Can't Feel My Face,The Weeknd
...,...,...,...
418,2019,Hot Girl Bummer,blackbear
419,2019,Bad Bad Bad,Young Thug
420,2019,INTRO,DaBaby
421,2019,Lost In The Fire,Gesaffelstein


In [3]:
# define genius token
token = 'Kh1Zx_dUoimcD2FeqfLTkZ9pCuDiA8rFa3nCT-Tu1ydQ15-vcwMgQ8aOrpnckFH0'

# define lyric scraper function
def get_lyrics(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + token}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    # Search for matches in the request response
    json = response.json()
    remote_song_info = None
    
    # check if hit
    for hit in json['response']['hits']:
        if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            break
    
    # Extract lyrics from URL if the song was found
    lyrics = np.nan
    if remote_song_info:
        song_url = remote_song_info['result']['url']
    
        # scrpae lyrics
        page = requests.get(song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()#.replace('\n', '')
    
    return lyrics

In [4]:
# get lyrics for songs
for i in range(top100.shape[0]):
    if i%100 == 0:
        print(i)
    title = top100.iloc[i,1]
    artist = top100.iloc[i,2]
    top100.loc[i,'Lyric'] = get_lyrics(title,artist)
    
top100

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420


,Year,Title,Artist,Lyric
0,2015,See You Again,Wiz Khalifa,\n\n[Chorus: Charlie Puth]\nIt's been a long d...
1,2015,Trap Queen,Fetty Wap,"\n\n[Intro]\nRGF productions\nRemy Boyz, yah-a..."
2,2015,Watch Me,Silento,"\n\n[Intro]\nWhip, nae nae\nWhip, whip, nae na..."
3,2015,The Hills,The Weeknd,\n\n[Intro]\nYeah\nYeah\nYeah\n\n[Verse 1]\nYo...
4,2015,Can't Feel My Face,The Weeknd,\n\n[Produced by Max Martin]\n\n[Verse 1]\nAnd...
...,...,...,...,...
418,2019,Hot Girl Bummer,blackbear,"\n\n[Intro]\nFuck you, and you, and you\nI hat..."
419,2019,Bad Bad Bad,Young Thug,\n\n[Intro]\nWheezy outta here\n\n[Chorus: You...
420,2019,INTRO,DaBaby,\n\n[Verse 1]\nThinkin' 'bout my grandmama and...
421,2019,Lost In The Fire,Gesaffelstein,\n\n[Verse 1]\nI wanna fuck you slow with the ...


In [18]:
# save and load data
top100.to_csv('top100.csv',index=False)
pd.read_csv('top100.csv')

,Year,Title,Artist,Lyric
0,2015,See You Again,Wiz Khalifa,\n\n[Chorus: Charlie Puth]\nIt's been a long d...
1,2015,Trap Queen,Fetty Wap,"\n\n[Intro]\nRGF productions\nRemy Boyz, yah-a..."
2,2015,Watch Me,Silento,"\n\n[Intro]\nWhip, nae nae\nWhip, whip, nae na..."
3,2015,The Hills,The Weeknd,\n\n[Intro]\nYeah\nYeah\nYeah\n\n[Verse 1]\nYo...
4,2015,Can't Feel My Face,The Weeknd,\n\n[Produced by Max Martin]\n\n[Verse 1]\nAnd...
...,...,...,...,...
418,2019,Hot Girl Bummer,blackbear,"\n\n[Intro]\nFuck you, and you, and you\nI hat..."
419,2019,Bad Bad Bad,Young Thug,\n\n[Intro]\nWheezy outta here\n\n[Chorus: You...
420,2019,INTRO,DaBaby,\n\n[Verse 1]\nThinkin' 'bout my grandmama and...
421,2019,Lost In The Fire,Gesaffelstein,\n\n[Verse 1]\nI wanna fuck you slow with the ...
